# 07 파이썬 정규표현식과 XML

## 07-1 Regular Expressions 정규표현식(정규식) 살펴보기
복잡한 문자열을 처리할 때 사용하는 기법. Python만의 고유 문법이 아니라 문자열을 처리하는 모든 곳에서 사용됨.

(ref: https://docs.python.org/3.4/howto/regex.html)


### 정규식 왜 필요?
문제:

주민등록번호를 포함하고 있는 텍스트가 있는데, 
이 텍스트에 포함한 모든 주민등록번호의 뒷자리를 \* 문자로 변경
1. 전체 텍스트를 공백 문자로 나눔(split)
2. 나누어진 단어들이 주민번호 형식인지 조사
3. 단어가 주민번호 형식이면 뒷자리 \*로 변환
4. 나누어진 단어 재조립

In [2]:
data = """
park 800905-1049118
kim  700905-1059119
"""

result = []
for line in data.split('\n'):
    word_result = []
    for word in line.split(' '):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + '-' '*******'
            
        word_result.append(word)
    result.append(' '.join(word_result))   # 요소 사이에 ' '추가하여 합침.
print('\n'.join(result))   # 요소 사이에 '\n'추가하여 합침.


park 800905-*******
kim  700905-*******



정규식을 사용하면 훨씬 간편하고 직관적인 코드가 된다.

In [3]:
import re

data = """
park 800905-1049118
kim  700905-1059119
"""

pat = re.compile("(\d{6})[-]\d{7}")  # 숫자 6개 + "-" 1개 + 숫자 7개
print(pat.sub("\g<1>-*******", data))


park 800905-*******
kim  700905-*******



## 07-2 정규 표현식 시작하기

### 정규 표현식의 기초, 메타 문자 (meta characters)

```
. ^ * + ? { } [ ] \ | ( )
```

**문자 클래스, character class []**

문자 클래스로 만들어진 정규식은 "\[ 와 \] 사이의 문자들과 매치"의 의미

ex) [abc]: "a, b, c 중 한 개의 문자와 매치
* "a": 일치하는 부분이 있으므로 매치
* "before": 일치하는 부분이 있으므로 매치
* "dude": 어느 하나도 포함하고 있지 않으므로 매치되지 않음

[form - to]

\[a-c\]: a 부터 c 사이 = [abc]

\[0-5\]: a 부터 c 사이 = [012345]

ex)
* [a-zA-Z]: 알파벳 모두
* [0-9]: 숫자

^ : 반대(not)의 의미
* [^0-9]: 숫자가 아닌 모든 문자

자주 사용하는 문자 클래스
* [0-9] & [a-zA-Z]
* \d : 숫자와 매치 = [0-9]
* \D : 숫자가 아닌것 과 매치 = [^0-9]
* \s : whitespace 문자와 매치 = [ \t\n\r\f\v] 맨 앞의 빈칸은 공백문자(space) 의미
* \S : whitespace 문자가 아닌 것과 매치 = [^ \t\n\r\f\v]
* \w : 문자와 숫자(alphanumeric)와 매치 = [a-zA-Z0-9]
* \W : 문자와 숫자가 아닌 문자와 매치 = [^a-zA-Z0-9]

#### Dot(.)
줄바꿈 문자인 \n 을 제외한 모든 문자와 매치

(re.DOTALL: \n 문자와도 매치)

```
a.b
```
"a + 모든문자 + b": a와 b라는 문자 사이에 어떤 문자가 들어가도 모두 매치
ex)
* "aab": 매치
* "a0b": 매치
* "abc": 비매치

주의: 문자 클래스 [ ] 내에 Dot(.) 메타 문자가 사용된다면 문자 "." 그대로를 의미
```
a[.]b
```
* "a.b": 매치
* "a0b": 비매치

#### 반복 (*)
반복을 의미하는 * 메타문자. 
```
ca*t
```
문자 a 가 0 부터 무한대로 반복될 수 있다는 의미
* "ct": 매치
* "cat": 매치
* "caaat": 매치

#### 반복 (+)
반복을 나타내는 또다른 문자 +
문자가 최소1번 이상 반복될 때 매치
```
ca+t
```
* "ct": 비매치
* "cat": 매치
* "caaat": 매치

#### 반복 ({m,n}, ?)
반복 횟수를 고정. {m, n} : 반복 횟수가 m부터 n까지인 것을 매치

{1,} = +

{0,} = *

**{m}**
```
ca{2}t
```
* cat: NO
* caat: YES


**{m, n}**
```
ca{2,5}t
```
* cat: NO
* caat: YES
* caaaaat: YES


**?**

{0,1}과 동일
```
ab?c
```
* abc: YES
* ac: YES

#### re 모듈, 파이썬에서 정규 표현식을 지원

re(regular expression)

In [9]:
import re
p = re.compile('ab*')

re.compile 을 이용하여 정규표현식을 컴파일

컴파일된 패턴객체(re.compile의 결과로 리턴되는 객체, p)를 이용하여 그 이후의 작업 수행 가능

* 패턴 = 정규식을 컴파일한 결과

#### 정규식을 이용한 문자열 검색
컴파일 된 패턴 객체는 4가지 메쏘드를 제공
* match(): 문자열의 처음부터 정규식과 매치되는지 조사
* search(): 문자열 전체를 검색하여 정규식과 매치되는지 조사
* findall(): 정규식과 매치되는 문자열(substring)을 리스트로 리턴
* finditer(): 정규식과 매치되는 문자열을 iterator 객체로 리턴

match, search는 정규식과 매치될 때에는 match 객체를 리턴, 매치되지 않는 경우 None 리턴

In [11]:
import re
p = re.compile('[a-z]+')  # 패턴 객체 생성: 영어소문자 1개 이상 반복

**match**

문자열의 처음부터 정규식과 매치되는지 조사


In [14]:
m = p.match("python")
print(m)  # 정규식 [a-z+]에 부합되므로 match 객체가 리턴

<_sre.SRE_Match object; span=(0, 6), match='python'>


In [15]:
m = p.match("3 python") 
print(m) # 처음부터 검색할 시 부합하지 않으므로 None 리턴

None


In [17]:
p = re.compile('\D+') # 숫자가 아닌 모든 문자 1번 이상 반복
m = p.match('string goes here')
if m:
    print('Match found:', m.group())
else:
    print('No match')

Match found: string goes here


**search**

문자열 전체를 검색하여 정규식과 매치되는지 조사. 

In [22]:
import re
p = re.compile('[a-z]+')  # 패턴 객체 생성: 영어소문자 1개 이상 반복
m = p.search("python")
print(m)  # 정규식 [a-z+]에 부합되므로 match 객체가 리턴

<_sre.SRE_Match object; span=(0, 6), match='python'>


In [23]:
m = p.search("3 python") 
print(m) # 두번 쨰 문자열은 부합하므로 match 객체 리턴

<_sre.SRE_Match object; span=(2, 8), match='python'>


**findall**

정규식과 매치되는 문자열(substring)을 리스트로 리턴

In [25]:
result = p.findall("life is too short")
print(result)

['life', 'is', 'too', 'short']


**finditer**

 정규식과 매치되는 문자열을 iterator 객체로 리턴

In [27]:
result = p.finditer("life is too short")
print(result)

In [28]:
for r in result: print(r)

<_sre.SRE_Match object; span=(0, 4), match='life'>
<_sre.SRE_Match object; span=(5, 7), match='is'>
<_sre.SRE_Match object; span=(8, 11), match='too'>
<_sre.SRE_Match object; span=(12, 17), match='short'>


#### match 객체의 메서드
* 어떤 문자열이 매치되었는가?
* 매치된 문자열의 인덱스는 어디서부터 어디까지인가?

종류
* group(): 매치된 문자열을 리턴
* start(): 매치된 문자열의 시작 위치를 리턴
* end(): 매치된 문자열의 끝 위치를 리턴
* span(): 매치된 문자열의 (시작, 끝)에 해당하는 tuple 리턴

In [29]:
m = p.match("python")
m.group()

'python'

In [34]:
m.start()  # match는 문자열 처음부터 검색하므로 항상 0

0

In [32]:
m.end()

6

In [33]:
m.span()

(0, 6)

In [35]:
m = p.search("3 python")
m.group()

'python'

In [36]:
m.start()

2

In [37]:
m.end()

8

In [38]:
m.span()

(2, 8)

#### 모듈 단위로 수행하기

In [40]:
m = re.match('[a-z]+', 'python')

### 컴파일 옵션
* DOTALL(S) - Dot(.)이 줄바꿈 문자를 포함하여 문자와 매치할 수 있도록
* IGNORECASE(I) - 대소문자에 관계없이 매치할 수 있도록
* MULTILINE(M) - 여러줄과 매치할 수 있도록 (^, $ 메타문자 사용과 관계가 있음)
* VERBOSE(X) - verbose 모드를 사용할 수 있도록 (정규식을 보기 편하게 만들 수 있고, 주석등을 가능하게 함)

#### DOTALL, S
. 메타 문자는 줄바꿈문자(\n)를 제외한 모든 문자와 매치되는 규칙

\n 문자도 포함하여 매치하고 싶다면 re.DOTALL 또는 re.S 옵션 사용

여러줄로 이루어진 문자열에서 \n 에 상관없이 검색하고자 할 경우 사용됨

In [49]:
import re
p = re.compile('a.b')
m = p.match('a\nb')
print(m)

None


In [44]:
p = re.compile('a.b', re.DOTALL)
m = p.match('a\nb')
print(m)

<_sre.SRE_Match object; span=(0, 3), match='a\nb'>


#### IGNORECASE, I
대소문자 구분없이 매치를 수행하도록

In [52]:
p = re.compile('[a-z]', re.I)
p.match('python')

<_sre.SRE_Match object; span=(0, 1), match='p'>

In [53]:
p.match('Python')

<_sre.SRE_Match object; span=(0, 1), match='P'>

In [54]:
p.match('PYTHON')

<_sre.SRE_Match object; span=(0, 1), match='P'>

#### MULTILINE, M
여러줄과 매치할 수 있도록 

* ^: 문자열의 처음을 의미
* $: 문자열의 마지막

ex) 

^python : 문자열의 처음은 항상 python으로 시작해야 매치됨.

python$ : 문자열의 마지막은 항상 python으로 끝나야 매치됨.

In [55]:
import re
p = re.compile("^python\s\w+") # python으로 시작하고 그 후에 whitespace, 그 후에 단어가 와야함

data = """python one
life is too short
python two
you need python
python tree"""

print(p.findall(data)) # python 이라는 문자열이 사용된 첫번쨰 라인만 매치가 됨.

['python one']


In [56]:
import re
p = re.compile("^python\s\w+", re.MULTILINE)

data = """python one
life is too short
python two
you need python
python tree"""

print(p.findall(data)) 

['python one', 'python two', 'python tree']


#### VERBOSE, X 
여려운 정규식을 주석 또는 라인 단위로 구분하고 첨삭 달 수 있게함.

In [57]:
charref = re.compile(r'&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);')

In [58]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

### 백슬래시(\) 문제
정규표현식을 파이썬에서 사용하려 할때 혼란을 줄 수 있다. 

ex) LaTex파일에서 \section이라는 문자열을 찾기 위한 정규식을 만든다고 가정할때
```
\section
```
\s 가 whtiespace 로 해석되어 의도한대로 매치가 안됨.

```
[ \t\n\r\f\v]eection
```
과 동일한 의미

```
\\section
```
의로 변경해야 함.

하지만, 이 또한 다른 문제 발견. 파이썬 문자열 리터럴 규칙에 의하여 \\ 이 \ 로 변경되어 \section이 전달됨 (파이썬만의 문제)

결국, 정규식 엔진에 \\문자를 전달하려면 파이썬은 \\\\, 백슬래시 4개가 필요

In [60]:
p = re.compile('\\\\section')

파이썬 정규식에 Raw String이 생겨남

In [62]:
p = re.compile(r'\\section') # r 문자를 선행하면, 정규식은 Raw String 규칙에 따름. 

## 07-3 강력한 정규 표현식의 세계로
다른 메타 문자, Group을 만드는 법, 전방 탐색 등을 배운다. 

### 메타분자
+, *, [], {} 매치가 진행될때 현재 매치되고 있는 문자열의 위치가 변경된다. 

문자열을 소모시키지 않는 (zero-width assertions) 메타 문자 에 대하여

#### |
"or"의 의미와 동일. A|B는 A 또는 B

In [64]:
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<_sre.SRE_Match object; span=(0, 4), match='Crow'>


#### ^
문자열의 맨 처음과 일치함을 의미. 

re.MULTILINE 사용할 경우, 여러줄의 문자열에서 각 라인의 처음과 일치하게됨.

In [67]:
print(re.search("^Life", "Life is too short"))

<_sre.SRE_Match object; span=(0, 4), match='Life'>


In [68]:
print(re.search("^Life", "My Life"))

None


#### $
문자열의 끝과 매치함

In [69]:
print(re.search('short$', 'Life is too short'))

<_sre.SRE_Match object; span=(12, 17), match='short'>


In [70]:
print(re.search('short$', 'Life is too short, you need python'))

None


'^','$'를 메타문자가 아닌 문자 그대로 매치하고 싶은 경우, [^], [$] 또는 \^, \$

#### \A
문자열의 처음과 매치됨, ^와 동일한 의미지만, 

re.MULTILINE 옵션을 사용할 경우, 라인과 상관없이 전체 문자열의 처음하고만 매치됨

#### \Z
문자열의 끝과 매치됨

re.MULTILINE 옵션을 사용할 경우, $와 달리 전체 문자열의 끝과 매치됨

#### \b
단어 구분자(Word boundary)이다. 보통 단어는 whitespace에 의해 구분됨

In [2]:
import re
p = re.compile(r'\bclass\b') # " class "와 매치됨.
print(p.search('no class at all'))

<_sre.SRE_Match object; span=(3, 8), match='class'>


In [3]:
print(p.search('one subclass is'))

None


주의: \b 메타 문자는 파이썬 리터럴 규칙에 의해 백스페이스 (Back Space)를 의미하므로 raw string을 사용해야함
    
```
r'\bclass\b'
```

#### \B 
\b 의 반대. whitespace로 구분된 단어가 아닌 경우

In [9]:
p = re.compile(r'\Bclass\B') # "...class..." 와 매치 
print(p.search('no class at all'))

None


In [7]:
print(p.search('the declassififed algorith'))

<_sre.SRE_Match object; span=(6, 11), match='class'>


In [8]:
print(p.search('one sublcass is'))

None


### 그룹핑
정규식 안에 매칭되는 그룹을 만들 수 있다!!!! ( )

In [36]:
p = re.compile('(ABC)+')
m = p.search('ABCABCABC OK?')
print(m)

<_sre.SRE_Match object; span=(0, 9), match='ABCABCABC'>


In [41]:
print(m.group(0))

ABCABCABC


In [42]:
print(m.group(1))

ABC


In [16]:
p = re.compile(r'\w+\s+\d+[-]\d+[-]\d+') # 이름 + " " + 전화번호
m = p.search("park 010-1234-1234")
print(m.group())

park 010-1234-1234


In [20]:
p = re.compile(r'(\w+)\s+\d+[-]\d+[-]\d+') # group(index) 메서드를 이용하여 그룹핑된 부분의 문자열만 뽑아낼 수 있다. 
m = p.search("park 010-1234-1234")
print(m.group(1))

park


* group(0): 매치된 전체 문자열
* group(1): 첫번째 그룹에 해당되는 문자열
* group(n): n번째 그룹에 해당되는 문자열

In [24]:
p = re.compile(r'(\w+)\s+(\d+[-]\d+[-]\d+)')
m = p.search("park 010-1234-1234")
print(m.group(1))

park


In [25]:
print(m.group(2))

010-1234-1234


In [33]:
p = re.compile(r'(\w+)\s+((\d+)[-]\d+[-]\d+)') 
# 그룹이 중첩되는 경우 바깥쪽부터 안쪽으로 인덱스 증가
m = p.search("park 010-1234-1234")
print(m.group(1))

park


In [34]:
print(m.group(2))

010-1234-1234


In [35]:
print(m.group(3))

010


**그룹핑된 문자열 재참조하기(Backreferences)**

재참조 문자

\1 : 그룹1의 단어 재참조

\2 : 그룹2의 단어 재참조

In [45]:
p = re.compile(r'(\b\w+)\s+\1')  # (그룹1) + " " + 그룹1과 동일한 단어
p.search('Paris in the the spring').group()

'the the'

#### 그룹핑된 문자열에 이름 붙이기, Named Groups
그룹을 인덱스가 아닌 이름으로 참조하게 하기
```
(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)
```

(\w+) 라는 그룹에 "name"이라는 이름을 붙였다. (?...)은 정규표현식의 확장구문. ... 은 anything을 의미

```
(?P<그룹명>...)
```

In [47]:
p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group("name"))

park


In [49]:
p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)')  # 그룹명 word를 이용하여 재참조, (?P=그룹명)
p.search('Paris in the the sping').group()

'the the'

### 전방 탐색, Lookahead Assertions
전방탐색 확장구문이 어려워보이지만, 매우 유용!

In [51]:
p = re.compile(".+:")
m = p.search("http://google.com")
print(m.group())

http:


정규식 ```.+:```과 일치하는 문자열 'http:'가 리턴.

':'를 출력하려면? 그룹핑 추가로 하지않고 한다면??

* 긍정형(positive) 전방 탐색 ```(?...)```: ...에 해당된 정규식과 매치되어야 하며 조건이 통과되어도 문자열이 소모되지 않는다. 
* 부정형(negative) 전방 탐색 ```(?!...)```: ...에 해당된 정규식과 매치되지 않아야 하며 조건이 통과되어도 문자열이 소모되지 않는다.

**긍정형 전방 탐색**

In [56]:
p = re.compile(".+(?=:)") # : >> (?=:)
m = p.search("http://google.com")
print(m.group())

http


기존 정규식과 검색에서 동일한 효과를 발휘하지만, 

: 에 해당되는 문자열이 정규식 엔진에 의해 소모되지 않아 (검색에는 포함되지만 검색결과에 제외됨) 검색 결과에서는 :이 제거된 후 리턴)

```
.*[.].*$   # 파일명 + '.' + 확장자 를 나타내는 정규식
```

정규식에 "확장자가 bat 인 파일은 제외해야 한다" 라는 조건 추가해보자
```
.*[.][^b].*$ 
```
확장자가 b라는 문자로 시작하면 안됨

```
*[.]([^b]..|.[^a].|...[^t])$
```
"|" 문자를 사용하여 확장자의 첫번째가 b가 아니거나, 두번째 문자가 a가 아니거나 세번째 문자가 t가 아닌 경우

하지만 이는 확장자의 문자 개수가 2개이 케이스를 포함하지 않음.

```
.+[.]([^b].?.?|.[^a]?.?|..?[^t]?)$
```
확장자의 문자 개수가 2개여도 통과되는 정규식

**부정형 전방 탐색**

계속해 늘어나는 긍정형 전방탐색 정규식을 구원해준다. 
```
.*[.](?!bat$).*$
```
확장자가 bat이 아닌 경우만 통과, bat라는 문자열이 있는 조사하는 과정에서 문자열이 소모되지 않으므로 bat가 아니라고 판단되면 그 이후 정규식 매칭이 진행
```
.*[.](?!bat$|exe$).*$
```
exe 역시 제외하라는 조건 추가

### 문자열
sub 메서드를 이용하면 정규식과 매치되는 부분을 다른 문자로 쉽게 바꿀 수 있음

In [58]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes') # 첫번째 입력인수: 바꿀 문자열(replacement)
# 두번째 입력 인수: 대상 문자열

'colour socks and colour shoes'

**sub 메서드와 유사한 subn 메서드**

sub과 동일한 기능을 하지만 리턴되는 결과를 tuple로 리턴. 

리턴된 tuple의 첫번째 요소는 변경된 문자열, 두번째 요소는 바꾸기가 발생한 횟수

In [59]:
p = re.compile('(blue|white|red)')
p.subn('colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

**sub 메서드 사용시 참조 구문 사용하기**

In [63]:
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub("\g<phone> \g<name>", "park 010-1234-1234")) # sub의 바꿀 문자열과 부분에 \g<그룹명>을 이용하면 정규식의 그룹명을 참조할수 있다. 

010-1234-1234 park


이름 + 전화번호 의 문자열을 전화번호 + 이름 으로 바꾸는 예.

In [66]:
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub("\g<2> \g<1>", "park 010-1234-1234")) # sub의 바꿀 문자열과 부분에 \g<그룹명>을 이용하면 정규식의 그룹명을 참조할수 있다. 

010-1234-1234 park


**sub 메서드의 입력 이수로 함수 넣기**

In [69]:
def hexrepl(match):
    "Return the hex string for a decimal number"
    value = int(match.group())
    return hex(value)

p = re.compile(r'\d+')
p.sub(hexrepl, 'Call 65490 for printing, 49152 for user code.')

'Call 0xffd2 for printing, 0xc000 for user code.'

### Greedy vs Non-Gready

In [70]:
s = '<html><heed><title>Title</title>'
len(s)
print(re.match('<.*>', s).span())

(0, 32)


In [71]:
print(re.match('<.*>', s).group())

<html><heed><title>Title</title>


메타 문자 *는 매우 greedy 하여 매치할 수 있는 최대 문자열인 ```<html><head><title>Title</title>``` 문자열 모두를 소모시킨다

어떻게 하면 ```<thml>```만 소모되도록 막을 수 있을까?

In [73]:
print(re.match('<.*?>', s).group())

<html>


non-greedy 문자인 ?은 *?, +?, ??, {m,n}?과 같이 사용하여 가능한 한 가장 최소한의 반복을 수행하도록 도와준다

## 07-4 파이썬으로 XML 처리하기
XML 처리를 위한 파이썬 라이브러리는 아주 많으며 다음 사이트 참조

http://wiki.python.org/moin/PythonXml

이 쳅터는 생략.